In [1]:
from spellchecker import SpellChecker

In [2]:
import pandas as pd

In [3]:
data = pd.read_excel("D:\CD\spell\VS_LOS_TBL_PROF_DETAILS.xlsx")

In [4]:
data.head()

,PROSPECTID,EMPLOYMENTTYPE,EMPLOYERNAME,Years in Curr Empl
0,3078CD0004450,SELF-EMPLOYED,SAHIL TELECOM,6
1,3078CD0004016,SELF-EMPLOYED,FRUIT AND VEGETABLES SHOP,6
2,3054CD0003543,SELF-EMPLOYED,SAHANARA STORES,5
3,3000CD0003431,SALARIED,SURESH BABU,4
4,3006CD0001457,SELF-EMPLOYED,Bazer store,4


In [5]:
data['stage_1'] = data.apply(lambda x: 'NUMBER' if isinstance(x['EMPLOYERNAME'], int) else x['EMPLOYERNAME'] , axis=1)

In [6]:
data['stage_1'] = data['stage_1'].str.lower()

In [30]:
data['stage_1'] = data['stage_1'].str.replace(" pvt ltd", " private limited")
data['stage_1'] = data['stage_1'].str.replace(" ltd", "limited")
data['stage_1'] = data['stage_1'].str.replace(" pvt ", " private")
data['stage_1'] = data['stage_1'].str.replace("pravit ", " private")

In [31]:
data['stage_1'] = data[['stage_1']].fillna(value='un known')

In [34]:
data['Employer_type'] = data.apply(lambda x: 'private limited' if " private limited" in  x['stage_1'] else 'no', axis=1)

In [10]:
#24222 -> private limited companies

In [11]:
data['stage_1'] = data['stage_1'].str.replace(" gov ", " government ")
data['stage_1'] = data['stage_1'].str.replace("gov", " government ")
data['stage_1'] = data['stage_1'].str.replace("gov ", " government ")
data['stage_1'] = data['stage_1'].str.replace(" gov", " government ")
data['stage_1'] = data['stage_1'].str.replace(" govt ", "government")
data['stage_1'] = data['stage_1'].str.replace("govt", "government")
data['stage_1'] = data['stage_1'].str.replace(" govt", "government")
data['stage_1'] = data['stage_1'].str.replace("govt ", "government")
data['stage_1'] = data['stage_1'].str.replace("gvt", "government")
data['stage_1'] = data['stage_1'].str.replace(" gvt ", "government")
data['stage_1'] = data['stage_1'].str.replace(" gvt", "government")
data['stage_1'] = data['stage_1'].str.replace("gvt ", "government")

In [38]:
data['Employer_type'] = data.apply(lambda x: 'government' if "government" in  x['stage_1'] else x['Employer_type'], axis=1)

In [39]:
data_df = data

In [40]:
data_df.head()

,PROSPECTID,EMPLOYMENTTYPE,EMPLOYERNAME,Years in Curr Empl,stage_1,Employer_type
0,3078CD0004450,SELF-EMPLOYED,SAHIL TELECOM,6,sahil telecom,no
1,3078CD0004016,SELF-EMPLOYED,FRUIT AND VEGETABLES SHOP,6,fruit and vegetables shop,no
2,3054CD0003543,SELF-EMPLOYED,SAHANARA STORES,5,sahanara stores,no
3,3000CD0003431,SALARIED,SURESH BABU,4,suresh babu,no
4,3006CD0001457,SELF-EMPLOYED,Bazer store,4,bazer store,no


In [41]:
uni_data  = pd.DataFrame(data_df.stage_1.unique(), columns=['stage_1'])

In [20]:
def spell_chk(row):
    spell = SpellChecker()
    test = row['stage_1'].split()
    for i in range(len(test)):
        if not isinstance(i, int):
            
            sp = spell.unknown([test[i]])
            if len(sp) == 0:
                test[i] = test[i] 
            else:
                test[i] = str(spell.correction(i))
        else:
            test[i] = test[i]
            
    print(row.name)
    row['spelling_corrected'] = ' '.join(test)
    return row #' '.join(test)

In [ ]:
from dask import dataframe as dd
from dask.multiprocessing import get
from multiprocessing import cpu_count

cores = cpu_count()-1

uni_data = dd.from_pandas(uni_data,npartitions=cores).\
   map_partitions(
      lambda df : df.apply(
         lambda x : spell_chk(x),axis=1)).\
   compute(scheduler=get)

0
0
1


In [23]:
uni_data

,stage_1,spelling_corrected
0,sahil telecom,sahil telecom
1,fruit and vegetables shop,fruit and vegetables shop
2,sahanara stores,sahanara stores
3,suresh babu,suresh babu
4,bazer store,bazer store
5,mechanic,mechanic
6,rice mill,rice mill
7,rashni lottery center,rashni lottery center
8,pravit teacher,pravit teacher
9,baba biswakarmakar service cen,baba biswakarmakar service cen


In [16]:
import os
os.environ["OPENBLAS_MAIN_FREE"] = "1"

import multiprocessing
import pandas as pd
import numpy as np
from multiprocessing import Pool
num_partitions = multiprocessing.cpu_count()
num_cores = multiprocessing.cpu_count()

In [17]:
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores-1)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
final_data = parallelize_dataframe(data_df, spell_chk)

In [7]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


'Io to the beach with my .'

In [9]:
words = set(nltk.corpus.words.words())

sent = "Gmmco Ltd happening"
" ".join(w for w in nltk.wordpunct_tokenize(sent) \
         if w.lower() in words or not w.isalpha())

'happening'

In [78]:
spell = SpellChecker()

# find those words that may be misspelled
misspelled = spell.unknown(['Bazer', 'store'])

for word in misspelled:
    print(word)
    # Get the one `most likely` answer
    print(spell.correction(word))
    # Get a list of `likely` options
    print(len(spell.candidates(word)))

bazer
baker
15


In [29]:
print("geeks pvt ltd".replace(" pvt ltd", " private limited")) 

geeks private limited


In [71]:
"geeks pvt ltd".split()

['geeks', 'pvt', 'ltd']

In [26]:
spell.correction('pravit')

'profit'

In [37]:
spell.candidates('pravite')

{'granite',
 'gravity',
 'kravitz',
 'parasite',
 'pavitt',
 'praise',
 'private',
 'privity',
 'provide',
 'ravine'}

In [80]:
spell.known(['Bazer', 'store'])

{'store'}

In [25]:
spell = SpellChecker()

test = "pravit SHOP".split()

for i in range(len(test)):
    sp = spell.unknown([test[i]])
    print(sp)
    if len(sp) == 0:
        test[i] = test[i] 
    else:
        test[i] = str(spell.correction(i))

test = ' '.join(test)
    
    

{'pravit'}


AttributeError: 'int' object has no attribute 'lower'

In [23]:
test

'FRUIT AND vegetables SHOP'

set()